In [5]:
import cv2
import mediapipe as mp
import tensorflow
import os
import uuid
import numpy as np
import time
from math import pi

In [6]:
mp_drawing=mp.solutions.drawing_utils
mp_hands=mp.solutions.hands

In [7]:
joint_list=[[0,5,8],[0,9,12],[0,13,16],[0,17,20],[0,2,4]]

In [8]:
def angle(v1, v2):
    angle = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
    angle=angle/pi*180
    return angle

In [ ]:
cap = cv2.VideoCapture(0)
prev_time=0
line=-1
index_pos=[[]]
new_line=False
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.8, max_num_hands=1) as hands:
    while cap.isOpened():
        #capture process video
        ret, frame = cap.read()
        image=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image=cv2.flip(image, 1)
        image.flags.writeable=False
        results=hands.process(image)
        image.flags.writeable=True
        image=cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #draw landmarks
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS)
    
            
            for hand in results.multi_hand_landmarks:      
                count=[0, 0, 0, 0, 0]
                for i, joint in enumerate(joint_list):
                    a=np.array([hand.landmark[joint[0]].x, hand.landmark[joint[0]].y, hand.landmark[joint[0]].z])
                    b=np.array([hand.landmark[joint[1]].x, hand.landmark[joint[1]].y, hand.landmark[joint[1]].z])
                    c=np.array([hand.landmark[joint[2]].x, hand.landmark[joint[2]].y, hand.landmark[joint[2]].z])
                    palm_joint=angle(a-b, c-b)
                    if (palm_joint>120):
                        count[i]=1
                    else: 
                        count[i]=0
                
                
                index=[int (hand.landmark[8].x*640), int(hand.landmark[8].y*480)]
                index[0]=int (2.5*(index[0]-320))
                index[1]=3*(index[1]-160)
                index=tuple(index)
                if count[0]:
                    cv2.circle(image, index, 12, (0, 0, 255), cv2.FILLED)
                if (count==[1, 1, 0, 0, 0]):
                    #index_pos.append(index)
                    
                    if new_line is False:
                        new_line=True
                        line+=1
                        index_pos.append([])
                    index_pos[line].append(index)
                else:
                    new_line=False
        for i in range(len(index_pos)):
            for j in range(len(index_pos[i])):
                if j>0:
                    cv2.line(image, index_pos[i][j-1], index_pos[i][j], (0, 0, 255), 6)
                        
        
        #calculate fps
        new_time=time.time()
        fps=1/(new_time-prev_time)
        prev_time=new_time
        fps=int(fps)
        fps=str(fps)
        fps='fps: '+fps
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(image, fps, (10, 50), font, 1, (100, 255, 0), 1, cv2.LINE_AA)
        
        
        cv2.imshow('Hand Tracking', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()